# Extract every 5th image from sequence of images

In [13]:
import os
import shutil

# This code extracts every 5th image frame from a video, so that the images for annotation has more variation.

from_path = "/Users/magnuswiik/prosjektoppgave_data/Masteroppgave_data/Videoer/GP010022/"
to_path = "/Users/magnuswiik/prosjektoppgave_data/Masteroppgave_data/Videoer/Annoteringsklar_GP010022/"

imgs = sorted(file for file in os.listdir(from_path) if file.endswith('.jpg'))

every_5th_img = imgs[::5]
    
for img in every_5th_img:
    shutil.copy(from_path + img, to_path)

# Extract subset of data from Labelme annotations

In [14]:
import os
import shutil

# This code extracts only the annotated image frames from an image folder and puts these images and annotation files in separate folders.

from_path = "/Users/magnuswiik/prosjektoppgave_data/Masteroppgave_data/Videoer/Annoteringsklar_GP010022/"
to_path = "/Users/magnuswiik/prosjektoppgave_data/Masteroppgave_data/Videoer/Annotert_GP010022/"

annots = sorted(file for file in os.listdir(from_path) if file.endswith('.json'))
imgs = sorted(file for file in os.listdir(from_path) if file.endswith('.jpg') and file[:-4] + '.json' in annots)

for annot in annots:
    shutil.copy(from_path + annot, to_path)
    
for img in imgs:
    shutil.copy(from_path + img, to_path)

# Change names on all files in a folder

In [15]:
import os

# This code adds the video identificator name to each file name in a folder.

path = "/Users/magnuswiik/prosjektoppgave_data/Masteroppgave_data/Videoer/Annotert_GP010022/"

annots = sorted(file for file in os.listdir(path) if file.endswith('.json'))
imgs = sorted(file for file in os.listdir(path) if file.endswith('.jpg') and file[:-4] + '.json' in annots)

for index, file in enumerate(annots):
    os.rename(os.path.join(path, file), os.path.join(path, "GP010022_" + file))

for index, file in enumerate(imgs):
    os.rename(os.path.join(path, file), os.path.join(path, "GP010022_" + file))

# Move Images to Images folder and Annotations to Boxes folder

In [21]:
import os
import shutil

# This code moves annotated images into a folder where it is ready for traning a Deep learning model.

from_path = "/Users/magnuswiik/prosjektoppgave_data/Masteroppgave_data/Videoer/Annotert_GP010022/"
to_path = "/Users/magnuswiik/prosjektoppgave_data/Masteroppgave_data/Helfisk_Deteksjonssett/"

annots = sorted(file for file in os.listdir(from_path) if file.endswith('.json'))
imgs = sorted(file for file in os.listdir(from_path) if file.endswith('.jpg') and file[:-4] + '.json' in annots)


for annot in annots:
    shutil.copy(from_path + annot, to_path + "Boxes/")
    
for img in imgs:
    shutil.copy(from_path + img, to_path + "Images/")

# Check if all annotation files contains shapes

In [3]:
import os, json

# This code checks if all annotations contain shapes. When using LabelMe it may happen that when you erase annotations, the annotated files are not removed
# but only the shapes are removed. This will create an error when training your model, since the model cannot learn anything from an empty annotation file.

data_path = "/Users/magnuswiik/prosjektoppgave_data/Masteroppgave_data/Laks_kroppsdeler_Deteksjonssett/"

items = os.listdir(data_path)

folders = [item for item in items if os.path.isdir(os.path.join(data_path, item)) and item != 'old']

for folder in folders:
    
    folder_path = os.path.join(data_path, folder)
    
    annots = sorted(file for file in os.listdir(folder_path) if file.endswith('.json'))
    
    for i in range(len(annots)):
        
        annot_path = os.path.join(folder_path, annots[i])

        with open(annot_path, 'r') as file:
            content = json.load(file)
            shapes = content['shapes']
            sh = len(shapes)
            if len(shapes) == 0:
                print(annots[i] + " is empty...")
                '''img_path = annot_path[:-4] + "jpg"
                os.remove(annot_path)
                os.remove(img_path)'''

# Crop salmon images using Labelme annotation files

In [22]:
import os, json
import cv2

# This code crops the annotated salmon out of the image frame.

from_path = "/Users/magnuswiik/prosjektoppgave_data/Masteroppgave_data/Helfisk_Deteksjonssett/"
to_path = "/Users/magnuswiik/prosjektoppgave_data/Masteroppgave_data/Helfisk_Landmark_Deteksjonssett/"

from_img = "GP020101_00012069.jpg"
from_json = "GP020101_00012069.json"

annots = sorted(file for file in os.listdir(from_path + "Boxes/") if file.endswith('.json'))
imgs = sorted(file for file in os.listdir(from_path + "Images/") if file.endswith('.jpg'))

for i in range(len(imgs)):

    with open(from_path + "Boxes/" + annots[i], 'r') as file:
        img = cv2.imread(from_path + "Images/" + imgs[i], cv2.IMREAD_COLOR)
        content = json.load(file)
        shapes = content['shapes']
        
        for shape in shapes:
            box = shape['points']
            x1 = int(box[0][0])
            y1 = int(box[0][1])
            x2 = int(box[1][0])
            y2 = int(box[1][1])
            id = shape['group_id']
            cropped_image = img[y1:y2, x1:x2, :]
            
            if not os.path.isdir(to_path + "fish" + str(id)):
                os.mkdir(to_path + "fish" + str(id))
                
            filename = to_path + 'fish' + str(id) + '/fish' + str(id) + "_"  + imgs[i]
            cv2.imwrite(filename, cropped_image)
            
            
    

# Crop bodypart images using Labelme annotations

In [7]:
import os, json
import cv2

# This code crops annotated body parts out of salmon images using LabelMe annotations.

from_path = "/Users/magnuswiik/prosjektoppgave_data/Masteroppgave_data/Laks_kroppsdeler_Deteksjonssett/"
to_path = "/Users/magnuswiik/prosjektoppgave_data/Masteroppgave_data/Laks_kroppsdeler_Identifikasjonssett/"

annots = []
imgs = []

for folder in os.listdir(from_path):
    if (not folder.startswith('.')) and os.path.isdir(os.path.join(from_path, folder)):
        for file in sorted(os.listdir(os.path.join(from_path, folder))):
            if file.endswith('.json'):
                annots.append(file)
        for file in sorted(os.listdir(os.path.join(from_path, folder))):
            if file.endswith(('.jpg', '.jpeg', '.png')) and file[:-4] + '.json' in annots:
                imgs.append(file)

for i in range(len(imgs)):
    with open(from_path + annots[i].split('_')[0] + "/" + annots[i], 'r') as file:
        img = cv2.imread(from_path + annots[i].split('_')[0] + "/" + imgs[i], cv2.IMREAD_COLOR)
        content = json.load(file)
        shapes = content['shapes']
        
        for shape in shapes:
            if shape['shape_type'] == 'rectangle':
                box = shape['points']
                x1 = int(box[0][0])
                y1 = int(box[0][1])
                x2 = int(box[1][0])
                y2 = int(box[1][1])
                fish = imgs[i].split('_')[0]
                landmark = shape['label']
                cropped_image = img[y1:y2, x1:x2, :]
                
                if not os.path.isdir(to_path):
                    os.makedirs(to_path)
                    
                filepath = os.path.join(to_path, fish + "_" + landmark + imgs[i][imgs[i].find("_"):])
                cv2.imwrite(filepath, cropped_image)
            

# Change filenames

In [2]:
import os

# This code changes filenames.

path = "/Users/magnuswiik/prosjektoppgave_data/Masteroppgave_data/Helfisk_Deteksjonssett/debug/edit/"

files = sorted(os.listdir(path))

for file in files:
    oldname = os.path.join(path,file)
    newname = os.path.join(path, "GP010101_" + file)
    os.rename(oldname, newname)